# Формат CSV
Comma separated values

In [1]:
import csv
import pandas as pd

import pyarrow as pa
import pyarrow.orc as orc

In [3]:
?pd.read_csv

Нужно читать большой файл
- iterator=True
- chunksize=100000

Используются нестандартные разделители разрядов и дробной части:
- thousands='.',
- decimal: 'str' = ',' --> 111.222,33

In [4]:
111_222.33

111.222,33

111222.33

In [2]:
df = pd.read_csv('data.csv', sep='\t', skiprows=10, nrows=10000)
df.head()

,date,category,campaign_id,slice,value
0,2020-11-21,59,26,3,688
1,2020-10-13,30,22,3,281
2,2020-07-24,80,24,9,266
3,2020-11-06,50,37,9,29
4,2020-02-21,70,50,7,25


А если в данных есть запятые?

In [5]:
sample = pd.DataFrame({
    'keywords': ['какая рыба вобла', 'фильмы, мотивирующие на уборку'],
    'date': ['2023-06-01', '2023-06-02']
})

sample

,keywords,date
0,какая рыба вобла,2023-06-01
1,"фильмы, мотивирующие на уборку",2023-06-02


In [6]:
sample.to_csv('sample.csv', index=False)

Простое чтение файлов

In [7]:
with open('sample.csv') as f:
    for line in f:
        line = line.strip().split(',')
        print(line)

['keywords', 'date']
['какая рыба вобла', '2023-06-01']
['"фильмы', ' мотивирующие на уборку"', '2023-06-02']


In [8]:
with open('sample.csv') as f:
    reader = csv.reader(f, delimiter=',')
    
    for line in reader:
        print(line)

['keywords', 'date']
['какая рыба вобла', '2023-06-01']
['фильмы, мотивирующие на уборку', '2023-06-02']


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1000 non-null   object
 1   category     1000 non-null   int64 
 2   campaign_id  1000 non-null   int64 
 3   slice        1000 non-null   int64 
 4   value        1000 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


# Формат JSON

In [9]:
import json

In [10]:
data = [
    {'date': '2023-01-01', 'value': 100}, 
    {'date': '2023-01-02', 'value': 200}
]

In [11]:
str(data)  # строка, но не формат JSON

"[{'date': '2023-01-01', 'value': 100}, {'date': '2023-01-02', 'value': 200}]"

In [12]:
json.dumps(data)  # а это уже формат JSON

'[{"date": "2023-01-01", "value": 100}, {"date": "2023-01-02", "value": 200}]'

In [13]:
with open('data.json', 'w') as f:
    f.write(json.dumps(data))

In [16]:
with open('data.json') as f:
    dict_ = json.loads(f.read())
    
dict_[0]['date']

'2023-01-01'

Пример данных от [API Яндекс Метрики](https://yandex.ru/dev/metrika/)

In [17]:
import requests

/Users/kbashevoy/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [18]:
params = {
    'id': 44147844,  # демо-счетчик
    'date1': '2023-01-01',  # начальная дата отчета
    'date2': '2023-01-07',  # конечная дата отчета
    'metrics': 'ym:s:visits',  # что будет в строках
    'dimensions': 'ym:s:date',  # что будет в столбцах
}

response = requests.get('https://api-metrika.yandex.ru/stat/v1/data', params=params, headers='')

In [19]:
response.status_code

200

In [21]:
response.json()['data']

[{'dimensions': [{'name': '2023-01-05'}], 'metrics': [521.0]},
 {'dimensions': [{'name': '2023-01-04'}], 'metrics': [512.0]},
 {'dimensions': [{'name': '2023-01-03'}], 'metrics': [476.0]},
 {'dimensions': [{'name': '2023-01-06'}], 'metrics': [433.0]},
 {'dimensions': [{'name': '2023-01-07'}], 'metrics': [432.0]},
 {'dimensions': [{'name': '2023-01-02'}], 'metrics': [395.0]},
 {'dimensions': [{'name': '2023-01-01'}], 'metrics': [246.0]}]

# Формат XML

In [22]:
import xml.etree.ElementTree as ET

In [23]:
data = """<?xml version="1.0"?>
<data>
    <rec>
        <date>2023-01-01</date>
        <value>100</value>
    </rec>
    <rec>
        <date>2023-01-02</date>
        <value>200</value>
    </rec>
    <rec>
        <date>2023-01-03</date>
        <value>300</value>
    </rec>
</data>
"""

In [24]:
root = ET.fromstring(data)

In [25]:
for rec in root.findall('rec'):
    print(rec.find('date').text, rec.find('value').text)

2023-01-01 100
2023-01-02 200
2023-01-03 300


Пример чтения из файла

In [26]:
root = ET.parse('data.xml')

In [27]:
for rec in root.findall('rec'):
    print(rec.find('date').text, rec.find('value').text)

2023-01-01 100
2023-01-02 200
2023-01-03 300


# Формат Avro
Установка [библиотеки avro](https://avro.apache.org/docs/1.11.1/getting-started-python/)
```
python3 -m pip install avro
```

In [28]:
import avro.schema

from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [29]:
schema = {
    "namespace": "example.avro",
    "type": "record",
    "name": "Data",
    "fields": [
        {"name": "date",  "type": ["string", "null"]},
        {"name": "value", "type": ["int", "null"]},
    ]
}

In [30]:
# этот вариант сохраняем в файл
json.dumps(schema)

'{"namespace": "example.avro", "type": "record", "name": "Data", "fields": [{"name": "date", "type": ["string", "null"]}, {"name": "value", "type": ["int", "null"]}]}'

In [ ]:
schema = avro.schema.parse(open("schema.avsc", "rb").read())
schema

In [ ]:
# создание файла

writer = DataFileWriter(open("data.avro", "wb"), DatumWriter(), schema)

writer.append({"date": "2023-01-01", "value": 100})
writer.append({"date": "2023-01-02", "value": 200})

writer.close()

In [31]:
reader = DataFileReader(open("data.avro", "rb"), DatumReader())

for rec in reader:
    print(rec)
    
reader.close()

{'date': '2023-01-01', 'value': 100}
{'date': '2023-01-02', 'value': 200}


# Формат parquet
[Документация](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html) метода to_parquet

In [33]:
df = pd.read_csv('data.csv', sep='\t')
df.head()

,date,category,campaign_id,slice,value
0,2020-11-21,59,26,3,688
1,2020-10-13,30,22,3,281
2,2020-07-24,80,24,9,266
3,2020-11-06,50,37,9,29
4,2020-02-21,70,50,7,25


In [ ]:
df.to_parquet('data.parquet', index=False)

In [34]:
pd.read_parquet('data.parquet').head()

,date,category,campaign_id,slice,value
0,2020-11-21,59,26,3,688
1,2020-10-13,30,22,3,281
2,2020-07-24,80,24,9,266
3,2020-11-06,50,37,9,29
4,2020-02-21,70,50,7,25


# Формат Orc

In [ ]:
table = pa.Table.from_pandas(df, preserve_index=False)
orc.write_table(table, 'data.orc')

In [35]:
# дока https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_orc.html
pd.read_orc('data.orc').head()

,date,category,campaign_id,slice,value
0,2020-11-21,59,26,3,688
1,2020-10-13,30,22,3,281
2,2020-07-24,80,24,9,266
3,2020-11-06,50,37,9,29
4,2020-02-21,70,50,7,25


# Pandas + Clickhouse
1. Ставим Docker desktop
2. Устанавливаем [образ](https://hub.docker.com/r/yandex/clickhouse-server/) Clickhouse
```
!docker run -d -p 0.0.0.0:8123:8123 --volume=/path/to/some/folder/on/disk/some_clickhouse_database:/var/lib/clickhouse --name some-clickhouse-server --ulimit nofile=262144:262144 yandex/clickhouse-server
```

Опция -p 0.0.0.0:8123:8123 открывает доступ к контейнеру по порту 8123 (иногда сразу его нет). В параметре volume пропишите папку на своем компьютере, где база будет хранить данные.


3. Проверьте наличие доступа к clickhouse в контейнере в браузере, открыв ссылку [localhost:8123](http://localhost:8123), должны увидеть Ok.

4. Берем открытые данные [Метрики](https://clickhouse.tech/docs/ru/getting-started/example-datasets/metrica/).

In [36]:
!ls

data.avro                      data.xml
data.csv                       files_formats_2023-07-06.ipynb
data.json                      sample.csv
data.orc                       schema.avsc
data.parquet


In [37]:
import clickhouse_connect

In [38]:
client = clickhouse_connect.get_client(host='localhost', username='', password='')

In [39]:
client.command('select count(*) from datasets.hits')

53243388

In [40]:
# размер данных

client.query_df("""
    SELECT 
        table,
        round(sum(data_uncompressed_bytes) / 1024 / 1024 / 1024, 1) as uncompressed_gb,
        round(sum(data_compressed_bytes) / 1024 / 1024 / 1024, 1) as compressed_gb,
        round(uncompressed_gb / compressed_gb, 1) as compression_rate

    FROM system.parts
    WHERE table = 'hits'
    GROUP BY table
""")

,table,uncompressed_gb,compressed_gb,compression_rate
0,hits,34.4,2.9,11.9


In [41]:
%%time

client.query_df('select EventDate, count(*) as hits from datasets.hits group by EventDate order by EventDate')

CPU times: user 2.23 ms, sys: 2.41 ms, total: 4.63 ms
Wall time: 148 ms


,EventDate,hits
0,2014-03-17,8441748
1,2014-03-18,8301948
2,2014-03-19,8434782
3,2014-03-20,8121738
4,2014-03-21,7474674
5,2014-03-22,6189552
6,2014-03-23,6278946


In [42]:
%%time

df = client.query_df('select TraficSourceID, EventDate, uniqExact(UserID) from datasets.hits group by TraficSourceID, EventDate')


CPU times: user 84.8 ms, sys: 4.24 ms, total: 89.1 ms
Wall time: 970 ms


In [43]:
df

,TraficSourceID,EventDate,uniqExact(UserID)
0,4,2014-03-21,528
1,6,2014-03-20,1294
2,2,2014-03-18,1606
3,-1,2014-03-21,21386
4,10,2014-03-21,3824
...,...,...,...
65,-1,2014-03-23,18574
66,10,2014-03-23,3434
67,6,2014-03-17,1410
68,0,2014-03-22,26854
